In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#time module to not get blocked
import time

#function to get all links to relevant jobs
def get_job_links(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #get last page number
    max_page_text = soup.select(".page-link")[-2].get_text()
    max_page_text = max_page_text.strip()
    max_page = int(max_page_text)
    
    links = []
    
    for i in range(1, max_page + 1):
        r = requests.get(f"{url}&page={i}")
        soup = BeautifulSoup(r.text, "html.parser")
        page_links = ["https://djinni.co" + a["href"] for a in soup.select(".job-item__title-link")]
        links.extend(page_links)
        time.sleep(3)
        
    return links

In [ ]:
url = "enter your url there"
links = get_job_links(url)

In [ ]:
def job_description(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #find if job is remote, hybrid or office-based
    try:
        arrangement_type = soup.find(class_="d-block font-weight-600", string=re.compile(r"(Office|Remote)", re.IGNORECASE))
        arrangement_type = arrangement_type.get_text()
    except AttributeError:
        arrangement_type = None
    #find if job is fulltime or part-time
    employment_type = soup.find(string=re.compile(r"Employment:"))
    if employment_type is not None:
        employment_type = employment_type.split("Employment: ")[1]
    
    #job domain
    domain = soup.find(string=re.compile(r"Domain:"))
    if domain is not None:
        domain = domain.split("Domain: ")[1]
    
    #English requirements
    language = soup.find(class_="fw-semibold", string=re.compile(r"English"))
    if language is not None:
        language = language.find_next("td").get_text()
    else:
        language = "Not Listed"
    
    #skills requirements
    requirements = soup.select(".row.gx-2.text-secondary.fs-5")
    if requirements:
        requirements = requirements[0].find("div", class_="col").get_text()
    else:
        requirements = "Not Listed"
    
    return [url, arrangement_type, employment_type, domain, language, requirements]

In [ ]:
job_descriptions = [] 
for link in links:
    a = job_description(link)
    job_descriptions.append(a)
    time.sleep(3)


In [ ]:
import pandas as pd
df = pd.DataFrame(job_descriptions, columns=["url", "arrangement_type", "employment_type", "domain", "English level", "requirements"])
df.to_csv("jobs.csv", index=False)